In [107]:
from interpreting_neurons_utils import *
import wandb
import pickle

- Look at python fuzzy logic libraries
- Experiment 1
    - Make the general setup, works for
        - different kinds of membership functions of output neurons
        - different kind of cost functions
        - different kind of input (Input are the fuzzy variables)
    - Define single fuzzy membership functions for all neurons
        - Look at histogram and do shit
        - Just do Negative, Positive
    - Use softmax for fuzzy input variables
    - use L1 sparcity constraint, extra for > 1
- ⇒
    - How good is MSE of neuron activations
    - Histogram of the weights
    - How realistic are the rules learned (are they the same that I would come up with)

- I think I need to do all the Fuzzy Logic shit by hand ...
- I think it makes sense to initialize all the fuzzy weights near 0.5

- First: not general?!
- Training Loop
    - Residual Stream
    - Proberesults (Softmax)
    - Fuzzy Inference -> Neuron Activation Pred (This is the step where things are learned)
    - Loss
    - Update

In [108]:
debug = False
EPSILON = 1e-6

In [148]:
@dataclass
class FuzzyTrainingArgs():
    # Which layer, and which positions in a game sequence to probe
    def __init__(self, run_number : int, layer : int, single_neuron=None):
        self.undersampling = True
        self.undersampling_factor = 1
        self.debug = False
        self.run_number = run_number
        self.layer: int = layer
        self.pos_start: int = 0
        self.pos_end: int = 59
        self.length: int = self.pos_end - self.pos_start
        self.max_epochs = 1
        self.single_neuron = single_neuron
        self.rules_count = 10
        self.neurons_count = 2048 if single_neuron is None else 1
        self.variables_count = 64 * (3 + 2 + 2)
        self.manual = False
        self.fuzzy_and = "mul"
        self.spacity_factor = 0.1
        self.only_0_1_factor = 0.1
        self.initialization = "ones"

        self.num_games_train: int = 250
        self.batch_size: int = 2

        self.lr: float = 1e-2
        self.betas: Tuple[float, float] = (0.9, 0.99)
        self.wd: float = 0.1

        self.run_name: str = f"Fuzzy_{self.run_number}"
        self.full_run_name: str = f"Fuzzy_L{self.layer}_{self.run_number}"

        # wnadb
        self.wandb_project: str = f"Train Fuzzy Logic Gates Test"

In [110]:
def get_centroid(trapezoid_metadata : List[Tuple[float, float, float, float]], trapezoid_heights : Float[Tensor, "tropezoid batch neurons"]):
    # This is the Center of Sums Method. Theres also the center of Gravity Method
    # TODO: get rid of the for loop, vectorize this
    trapezoids, batches, neurons = trapezoid_heights.shape
    member_ship_funcs = []
    numerrator = t.zeros((batches, neurons)).to(device)
    denomenator = t.zeros((batches, neurons)).to(device)
    for i, (a, b, c, d) in enumerate(trapezoid_metadata):
        trapezoid_height = trapezoid_heights[i]
        trapezoid_height = t.max(trapezoid_height, t.tensor(EPSILON).to(device))
        # b_new = a + trapezoid_height * (b-a)
        # c_new = d - trapezoid_height * (d-c)
        b_new = a + trapezoid_height * (b-a)
        c_new = d - trapezoid_height * (d-c)
        long_base = d - a
        short_base = c_new - b_new
        # individual_centroid = ((long_base + 2 * short_base) * trapezoid_height) / (3*(long_base + short_base)) + a
        # individual_centroid = a + ((b_new - a)**2 + 3*(c_new - b_new)**2 + (d - c_new)**2) / (3 * (- a - b_new + c_new + d))
        d_1 = b_new-a
        d_2 = c_new - b_new
        d_3 = d - c_new
        individual_centroid = ((a * d_1 / 2) + (d_1**2 / 6) + (b_new * d_2) + (d_2**2 / 2) + (d * d_3 / 2) - (d_3**2 / 6)) / ((d_1 / 2) + d_2 + (d_3 / 2))
        # individual_centroid = ((a + d_1/3) * (d_1 * trapezoid_height)/2 + (b_new + d_2/2) * (d_2 * trapezoid_height) + (d - d_3/3) * (d_3 * trapezoid_height)/2) / ((d_1 * trapezoid_height)/2 + d_2 * trapezoid_height + (d_3 * trapezoid_height)/2)
        individual_area = (long_base + short_base) / 2 * trapezoid_height
        numerrator += individual_centroid * individual_area
        denomenator += individual_area
    return numerrator / denomenator

In [111]:
import collections, gc, resource, torch
def debug_memory():
    print('maxrss = {}'.format(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss))
    tensors = collections.Counter(
        (str(o.device), str(o.dtype), tuple(o.shape))
        for o in gc.get_objects()
        if torch.is_tensor(o)
    )
    for line in sorted(tensors.items()):
        print('{}\t{}'.format(*line))

In [112]:
plot_game(focus_games_string, 13, end_move=16)

In [161]:
"""layer = 1
module = f"blocks.{layer}.ln2.hook_normalized"
neuron = 426
neurons_count = 2048
variables_count = 64 * (3 + 2 + 2)
rules_count = 10

neuron_negative_func = MemberShipFunc(-5, -3, 0, 0.5)
neuron_positive_func = MemberShipFunc(0, 0.5, 3, 5)"""

# neuron_membership_funcs = [(-0.3, 0, 0, 0.1), (0, 0.1, 1, 3)]
# neuron_membership_funcs = [(-0.2, -0.13, -0.13, -0.04), (2.7, 2.7, 3.7, 3.7)]
neuron_membership_funcs = [(-0.26, -0.26, 0.04, 0.04), (2.7, 3.2, 3.2, 3.7)]

# TODO: First make funcitons narrow, then gerenerell
def variables_to_tensor(input_variables : Dict[str, Tuple]) -> Float[Tensor, "variables"]:
    out = t.Tensor((64 * (3 + 2 + 2)))
    i = 0
    for _, value in input_variables.items():
        for j in range(len(value)):
            out[i] = value[j]
            i += 1
    return out

def tensor_to_variables(input_tensor : Float[Tensor, "variables"]) -> Dict[str, Tuple]:
    pass

def compute_fuzzy_out(fuzzy_weights, input_variables, neurons, rules):
    fuzzy_weights_r = einops.repeat(fuzzy_weights, "neurons rules variables -> neurons rules variables")
    input_variables_r = einops.repeat(input_variables, "variables -> neurons rules variables", neurons=neurons, rules=rules)
    updated_variables : Float[Tensor, "neuron rules variables"] = 1 - fuzzy_weights_r * (1 - input_variables_r)
    # fuzzy_mid = einops.reduce(updated_variables, "pos neurons rules variables -> pos neurons rules", "min")  # AND Gate
    # fuzzy_out = einops.reduce(fuzzy_mid, "pos neurons rules -> pos neurons", "max")  # OR Gate
    updated_variables = updated_variables.min(dim=-1).values  # AND Gate
    updated_variables = updated_variables.max(dim=-1).values  # OR Gate
    return updated_variables

def compute_fuzzy_mid(fuzzy_weights, input_variables, v, batch, neurons, rules):
    return 1 - einops.repeat(fuzzy_weights[:, :, v], "neurons rules -> batch neurons rules", batch=batch) * (1 - einops.repeat(input_variables[:, :, v], "batch -> batch neurons rules", neurons=neurons, rules=rules))

def compute_fuzzy_out_new(fuzzy_weights, input_variables):
    neurons, rules, variables = fuzzy_weights.shape
    batch, variables = input_variables.shape
    # fuzzy_weights_r = einops.repeat(fuzzy_weights, "neurons rules variables -> pos neurons rules variables")
    # input_variables_r = einops.repeat(input_variables, "batch pos variables -> pos neurons rules variables")
    fuzzy_mid = t.zeros((batch, neurons, rules)).to(device)
    for v in range(variables):
        if debug:
            print(f"Fuzzy Logic started for variable {v}")
            debug_memory()
        fuzzy_mid = t.min(fuzzy_mid, compute_fuzzy_mid(fuzzy_weights, input_variables, v, batch, neurons, rules))
    # fuzzy_mid = einops.reduce(updated_variables, "pos neurons rules variables -> pos neurons rules", "min")  # AND Gate
    # fuzzy_out = einops.reduce(fuzzy_mid, "pos neurons rules -> pos neurons", "max")  # OR Gate
    updated_variables = updated_variables.min(dim=-1).values  # AND Gate
    updated_variables = updated_variables.max(dim=-1).values  # OR Gate
    return updated_variables


def perform_fuzzy_logic(args : FuzzyTrainingArgs, input_variables : Float[Tensor, "batch variables"], fuzzy_weights : Float[Tensor, "neuron rules variables"]) -> Dict[str, Tuple]:
    batch, variables = input_variables.shape
    neurons, rules, _ = fuzzy_weights.shape
    if neurons == 1:
        fuzzy_weights_r = einops.repeat(fuzzy_weights, "neurons rules variables -> batch neurons rules variables", batch=batch)
        input_variables_r = einops.repeat(input_variables, "batch variables -> batch neurons rules variables", neurons=neurons, rules=rules)
        updated_variables : Float[Tensor, "batch neuron rules variables"] = 1 - fuzzy_weights_r * (1 - input_variables_r)
        if args.fuzzy_and == "mul":
            fuzzy_mid = updated_variables.prod(dim=-1) # AND Gate
        else:
            fuzzy_mid = einops.reduce(updated_variables, "batch neurons rules variables -> batch neurons rules", "min")  # AND Gate
        if args.rules_count == 1:
            fuzzy_out = fuzzy_mid.squeeze(dim=-1)
        else:
            fuzzy_out = einops.reduce(fuzzy_mid, "batch neurons rules -> batch neurons", "max")  # OR Gate
        return fuzzy_out
    fuzzy_out_complete = []
    for b in range(batch):
        if debug:
            print(f"Fuzzy Logic started for batch {b}")
            debug_memory()
        fuzzy_out = compute_fuzzy_out(fuzzy_weights, input_variables[b], neurons, rules)
        fuzzy_out_complete.append(fuzzy_out)
    fuzzy_out_complete = t.stack(fuzzy_out_complete, dim=0)
    # fuzzy_out_complete = compute_fuzzy_out_new(fuzzy_weights, input_variables)
    return fuzzy_out_complete

def perform_defuzzification(neuron_positive : Float[Tensor, "batch neurons"]) -> Float[Tensor, "batch neurons"]:
    # I need the fuzzy membership funcitions for the neurons and some algorithm
    if debug:
        print("Defuzzification started")
    neuron_negative = 1 - neuron_positive
    # batch, neurons = neuron_positive.shape
    neuron_activation = get_centroid(neuron_membership_funcs, t.stack([neuron_negative, neuron_positive], dim=0))
    if debug:
        print("Defuzzification ended")
    return neuron_activation

def full_fuzzy_inference(args : FuzzyTrainingArgs, input_variables : Float[Tensor, "batch variables"], fuzzy_weights : Float[Tensor, "neuron rules variables"]) -> Dict[str, float]: # Probe Results -> Neuron Activations
    """
    Perform fuzzy inference on the input variables.
    """
    # input_variables = input_variables_to_tensor(input_variables)
    # 1. Turn Fuzzy Variable into another Fuzzy Variable using the learned logical rules
    # The shape here will change with better neuron membership functions
    fuzzy_out : Float[Tensor, "batch neurons"] = perform_fuzzy_logic(args, input_variables, fuzzy_weights)
    # 2. Turn the Fuzzy Variable into a crisp value using defuzzification
    neuron_activations : Float[Tensor, "batch neurons"] = perform_defuzzification(fuzzy_out)
    return neuron_activations

def get_variables(resid_mid : Float[Tensor, "batch d_model"], layer) -> Float[Tensor, "batch variables"]:
    probe_results_all = []
    for probe_name in ["linear", "flipped", "placed"]:
        probe = get_probe(layer, probe_name, "mid")[0]
        probe_result = einops.einsum(resid_mid, probe, "batch d_model, d_model rows cols options -> batch rows cols options").softmax(dim=-1)
        probe_result = einops.rearrange(probe_result, "batch rows cols options -> batch (rows cols options)")
        probe_results_all.append(probe_result)
    probe_results_all = t.cat(probe_results_all, dim=-1)
    return probe_results_all

def init_weights(args : FuzzyTrainingArgs) -> Float[Tensor, "neurons rules variables"]:
    if args.manual:
        fuzzy_weights = t.zeros(size=(args.neurons_count, args.rules_count, 8, 8, 2), requires_grad=False).to(device)
        """fuzzy_weights[0, 0, *label_to_tuple("E2"), 5] = 1
        # fuzzy_weights[0, 0, *label_to_tuple("E3"), 3] = 1
        # fuzzy_weights[0, 0, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 1, *label_to_tuple("F3"), 5] = 1
        # fuzzy_weights[0, 1, *label_to_tuple("E3"), 3] = 1
        # fuzzy_weights[0, 1, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 2, *label_to_tuple("C5"), 5] = 1
        # fuzzy_weights[0, 2, *label_to_tuple("E3"), 3] = 1
        # fuzzy_weights[0, 2, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 3, *label_to_tuple("B6"), 5] = 1
        # fuzzy_weights[0, 3, *label_to_tuple("E3"), 3] = 1
        # fuzzy_weights[0, 3, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 4, *label_to_tuple("F5"), 5] = 1
        fuzzy_weights[0, 4, *label_to_tuple("E4"), YOURS] = 1
        fuzzy_weights[0, 4, *label_to_tuple("F3"), YOURS] = 1
        fuzzy_weights[0, 4, *label_to_tuple("E2"), YOURS] = 1
        fuzzy_weights[0, 4, *label_to_tuple("F1"), YOURS] = 1"""
        fuzzy_weights[0, 0, *label_to_tuple("B4"), PLACED] = 1

        fuzzy_weights = einops.rearrange(fuzzy_weights, "neurons rules rows cols options -> neurons rules (rows cols options)")
        fuzzy_weights = t.cat([t.zeros(size=(args.neurons_count, args.rules_count, 64 * (3 + 2)), requires_grad=False).to(device), fuzzy_weights], dim=-1)
        print(fuzzy_weights.shape)
        fuzzy_weights.requires_grad = True
        return fuzzy_weights
    
    # fuzzy_weights = t.randn(size=(neurons_count, rules_count, variables_count), requires_grad=False).to(device)
    # fuzzy_weights = fuzzy_weights.uniform_(-1, 1)
    if args.initialization == "ones":
        fuzzy_weights = t.ones(size=(args.neurons_count, args.rules_count, args.variables_count), requires_grad=False).to(device)
    elif args.initialization == "zeros":
        fuzzy_weights = t.zeros(size=(args.neurons_count, args.rules_count, args.variables_count), requires_grad=False).to(device)
    elif args.initialization == "uniform":
        fuzzy_weights = t.randn(size=(args.neurons_count, args.rules_count, args.variables_count), requires_grad=False).to(device)
        # fuzzy_weights = fuzzy_weights.uniform_(-1, 1) + EPSILON
    else:
        raise NotImplementedError
    fuzzy_weights.requires_grad = True
    return fuzzy_weights

# Try one inference
"""variables = get_variables(focus_cache[module])
variables = variables[:2, :]
weights = init_weights()
neuron_activations = full_fuzzy_inference(variables, weights)
print(neuron_activations[0, 0, neuron])"""

'variables = get_variables(focus_cache[module])\nvariables = variables[:2, :]\nweights = init_weights()\nneuron_activations = full_fuzzy_inference(variables, weights)\nprint(neuron_activations[0, 0, neuron])'

In [114]:
# TODO: Make undersampling good so that the batchsize is constant ...

In [172]:
def get_rules_dict(weights : Float[Tensor, "neurons rules variables"], neuron : int = 0, thresh : float = 0.1):
    # TODO: Why did I put all these dimensions together in the first place? I guess it is easier to understand this way
    weights = t.sigmoid(weights)
    variables_expanded = {
        "linear" : einops.rearrange(weights[:, :, :64*3], "neurons rules (rows cols options) -> neurons rules rows cols options", rows=8, cols=8, options=3)[neuron],
        "flipped" : einops.rearrange(weights[:, :, 64*3:64*5], "neurons rules (rows cols options) -> neurons rules rows cols options", rows=8, cols=8, options=2)[neuron],
        "placed" : einops.rearrange(weights[:, :, 64*5:], "neurons rules (rows cols options) -> neurons rules rows cols options", rows=8, cols=8, options=2)[neuron],
    }
    out_dict = {}
    for probe_name, variables in variables_expanded.items():
        rules, rows, cols, options = variables.shape
        for rule in range(rules):
            for row in range(rows):
                for col in range(cols):
                    for option in range(options):
                        weight = variables[rule, row, col, option]
                        if weight > thresh or thresh == -1:
                            label = tuple_to_label((row, col))
                            option_str = get_direction_str(probe_name, option)
                            out_dict[(rule,label,probe_name, option_str)] = weight.item()
                            # print(f"Rule {rule}, {tuple_to_label((row, col))} is {probe_name} {option}: {weight}")
    return out_dict

In [186]:
EPSILON = 1e-6
class FuzzyTrainer:
    def __init__(self, model: HookedTransformer, args: FuzzyTrainingArgs, train_base_resid):
        self.model = model
        self.args = args
        self.probe_names = {
            "linear" : ["empty", "yours", "mine"],
            "flipped" : ["flipped", "not_flipped"],
            "placed" : ["placed", "not_placed"],
        }
        self.weights : Float[Tensor, "neurons rules variables"] = init_weights(self.args)

    def training_step(self, indices: Int[Tensor, "game_idx"], train_or_val="train") -> t.Tensor:
        # Get the game sequences and convert them to state stacks
        games_int = board_seqs_int[indices.cpu()]
        games_str = board_seqs_string[indices.cpu()]
        batch_size = self.args.batch_size
        game_len = self.args.length
        # options = self.args.options
        d_model = 512

        # games_int = tensor of game sequences, each of length 60
        # This is the input to our model
        assert isinstance(games_int, Int[Tensor, f"batch={batch_size} full_game_len=60"])

        # SOLUTION
        with t.inference_mode():
            _, cache = model.run_with_cache(
                games_int[:, :-1].to(device),
                return_type=None,
                names_filter=lambda name: name == f"blocks.{self.args.layer}.ln2.hook_normalized" or name == f"blocks.{self.args.layer}.mlp.hook_post"
            )
            # TODO: Make undersampling work for generell case
            resid_layer_norm = cache[f"blocks.{self.args.layer}.ln2.hook_normalized"][:, self.args.pos_start: self.args.pos_end]
            neuron_activations : Float[Tensor, "batch pos neurons"] = cache["mlp_post", self.args.layer][:, self.args.pos_start: self.args.pos_end]
            resid_layer_norm = einops.rearrange(resid_layer_norm, "batch pos d_model -> (batch pos) d_model")
            neuron_activations = einops.rearrange(neuron_activations, "batch pos neurons -> (batch pos) neurons")
            if self.args.single_neuron is not None:
                neuron_activations = neuron_activations[:, self.args.single_neuron]
                # neuron_activations = einops.rearrange(neuron_activations, "batch pos neurons -> (batch pos neurons)")
                if self.args.undersampling:
                    count_positive = (neuron_activations > 0).sum()
                    count_all = len(neuron_activations)
                    neuron_activations_ideces = neuron_activations.argsort(descending=True)
                    positive_indeces = neuron_activations_ideces[:count_positive]
                    negative_indeces = neuron_activations_ideces[t.randperm(count_all - count_positive).to(device)[:int(count_positive * self.args.undersampling_factor)] + count_positive]
                    indeces = t.cat([positive_indeces, negative_indeces])
                    neuron_activations = neuron_activations[indeces]
                    resid_layer_norm = resid_layer_norm[indeces]
            # resid = cache["resid_mid", layer][:, self.args.pos_start: self.args.pos_end]
        resid_layer_norm = resid_layer_norm.clone().detach().to(device)
        variables : Float[Tensor, "batch variables"] = get_variables(resid_layer_norm, self.args.layer)
        sigmoid_weights = t.sigmoid(self.weights)
        neuron_activations_pred : Float[Tensor, "batch neurons"] = full_fuzzy_inference(self.args, variables, sigmoid_weights)
        if self.args.single_neuron is not None:
            neuron_activations_pred = neuron_activations_pred[:, 0]

        mse : Float[Tensor, "batch neurons"] = (neuron_activations - neuron_activations_pred)**2
        mse_loss = mse.mean()

        # TODO: Create a fancy loss function that everything at ones (Forces the weights to be either 0 or 1, but allowing transitions between these two)
        # This is currently a placeholder
        sparcity_loss = 1 * sigmoid_weights.abs().mean()
        spacity_loss2 = 2 * (sigmoid_weights.clamp(max=0).abs() + (sigmoid_weights.clamp(min=0, max=0.5) - 0).abs() + ((sigmoid_weights - 1).clamp(min=-0.5, max=0)).abs() + ((sigmoid_weights - 1).clamp(min=0)).abs()).mean()
        loss = mse_loss + self.args.sparcity_factor * sparcity_loss + self.args.only_0_1_factor * spacity_loss2
        
        if train_or_val == "train" and self.step % 10 == 0 and not self.args.debug:
            wandb.log({f"{train_or_val}_loss": loss.item()}, step=self.step)
        if self.step % 50 == 0 and self.args.debug: 
            rules_dict = get_rules_dict(self.weights, thresh=0.05)
            print(self.step)
            print(rules_dict)
        self.step += 1

        return loss
        
    def save_weights(self, object, path):
        folder_path = "/".join(path.split("/")[:-1])
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        with open(path, "wb") as f:
            pickle.dump(object, f)

    def shuffle_indices(self, start, num_games):
        '''
        Helper function
        '''
        n_indices = num_games - (num_games % self.args.batch_size)
        full_train_indices = t.randperm(num_games)[:n_indices] + start
        full_train_indices = einops.rearrange(full_train_indices, "(batch_idx game_idx) -> batch_idx game_idx", game_idx=self.args.batch_size)
        return full_train_indices
    
    def shuffle_training_indices(self):
        '''
        Returns the tensors you'll use to index into the training data.
        '''
        return self.shuffle_indices(0, self.args.num_games_train)

    def train(self):
        print(f"Training Probe: {self.args.full_run_name}")
        self.step = 0
        if not self.args.debug:
            wandb.login()
            wandb.init(project=self.args.wandb_project, name=self.args.full_run_name, config=self.args)

        optimizer = t.optim.AdamW([self.weights], lr=self.args.lr, betas=self.args.betas)

        for epoch in range(self.args.max_epochs):
            full_train_indices = self.shuffle_training_indices()
            progress_bar_train = tqdm(full_train_indices)
            for indices in progress_bar_train:
                loss = self.training_step(indices)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                progress_bar_train.set_description(f"Train_Loss = {loss:.4f}")

        if not self.args.debug:
            wandb.finish()
            self.save_weights(self.weights, f"fuzzy_weights/_{self.args.full_run_name}.pth")
        print("Probe Trained and Saved")

In [203]:
# set torch to random seed and print the seed
# It's the same every time because of the seed (What happens at 60000?)

t.manual_seed(42)
run_number = 0
neuron = 485 # 368
manual = False

if manual:
    args = FuzzyTrainingArgs(run_number, layer=0, single_neuron=neuron)
    args.num_games_train = 12800
    args.fuzzy_and = "min"
    args.batch_size = 128
    args.undersampling_factor = 1
    args.rules_count = 1
    args.manual = True
    args.debug = True
    trainer = FuzzyTrainer(model, args, train_base_resid=False)
    trainer.train()
else:
    args = FuzzyTrainingArgs(run_number, layer=0, single_neuron=neuron)
    args.num_games_train = 1000
    args.batch_size = 1
    args.rules_count = 2
    args.undersampling_factor = 1 # ONlY Positive counts ...
    args.sparcity_factor = 10 # 0.2
    args.only_0_1_factor = 0 # 0.1
    args.lr = 1e-1
    args.fuzzy_and = "mul"
    args.initialization = "uniform"
    args.manual = False
    args.debug = True
    trainer = FuzzyTrainer(model, args, train_base_resid=False)
    trainer.train()

Training Probe: Fuzzy_L0_0


  0%|          | 0/1000 [00:00<?, ?it/s]

0
{(0, 'A0', 'linear', 'empty'): 0.8729075789451599, (0, 'A0', 'linear', 'yours'): 0.8156703114509583, (0, 'A0', 'linear', 'mine'): 0.7110968828201294, (0, 'A1', 'linear', 'empty'): 0.10856136679649353, (0, 'A1', 'linear', 'yours'): 0.6633856296539307, (0, 'A1', 'linear', 'mine'): 0.22538696229457855, (0, 'A2', 'linear', 'empty'): 0.4892348051071167, (0, 'A2', 'linear', 'yours'): 0.16733035445213318, (0, 'A2', 'linear', 'mine'): 0.32035622000694275, (0, 'A3', 'linear', 'empty'): 0.8387184143066406, (0, 'A3', 'linear', 'yours'): 0.4031207859516144, (0, 'A3', 'linear', 'mine'): 0.19724434614181519, (0, 'A4', 'linear', 'empty'): 0.325659841299057, (0, 'A4', 'linear', 'yours'): 0.36367931962013245, (0, 'A4', 'linear', 'mine'): 0.3167303502559662, (0, 'A5', 'linear', 'empty'): 0.6818844079971313, (0, 'A5', 'linear', 'yours'): 0.8378499150276184, (0, 'A5', 'linear', 'mine'): 0.46018514037132263, (0, 'A6', 'linear', 'empty'): 0.378152459859848, (0, 'A6', 'linear', 'yours'): 0.6081611514091492

In [ ]:
x = t.tensor([1.0, 2.0, 3.0, 0.5, 4.0], requires_grad=True)

# Compute the minimum value
min_value = x.min()

# Perform the backward pass
min_value.backward()

# Print the gradients
print(f"Tensor x: {x}")
print(f"Minimum value: {min_value.item()}")
print(f"Gradients: {x.grad}")

Tensor x: tensor([1.0000, 2.0000, 3.0000, 0.5000, 4.0000], requires_grad=True)
Minimum value: 0.5
Gradients: tensor([0., 0., 0., 1., 0.])


In [ ]:
# TODO: Iterate until the manual weights get to the minimum loss
# TODO: Add a constraint, that weights aren't between 0 and 1
# TODO: Add Very Positive: I think this could make everything nicer
# TODO: Try setting 20 random weights to one and see if the loss is greater then my current manual weights ...

In [193]:
rules_dict2 = get_rules_dict(trainer.weights, neuron=0)
print(rules_dict2)
print(rules_dict2[(0, "B4", "placed", "placed")])
print(min(rules_dict2.values()))
print(max(rules_dict2.values()))

{(0, 'B4', 'placed', 'placed'): 0.9966287016868591}
0.9966287016868591
0.9966287016868591
0.9966287016868591
